In [229]:
import re, glob
import pandas
import pystain

In [230]:
#subject_ids = [13044, 13046, 13054, 13058, 13060, 13061, 13072, 13074, 13077, 15052]
subject_ids = [13044, 13046, 13054, 13058, 13060, 13061, 13072, 13074]

In [231]:
from matplotlib.backends.backend_pdf import PdfPages
import skimage
from skimage import io
import nibabel as nb

In [232]:
results = []

for subject_id in subject_ids:
    print subject_id
    
    year = '20' + str(subject_id)[:2]
    n = str(subject_id)[2:]
    
    im = skimage.io.imread('/home/public/HISTO_TO_MRI_1/Blockface_stacks/{subject_id}/{subject_id}_rt.tif'.format(**locals()))
    mri_slices = pandas.read_csv('/home/public/HISTO_TO_MRI_1/Blockface_stacks/True_slice_distance/{n}_{year}_3removed.txt'.format(**locals()), header=None, names=['slice'])
   
    ds = pystain.StainDataset(subject_id)
    
    if subject_id == None:
        r = {'subject_id':subject_id, 'bf_tif_x':im.shape[1], 'bf_tif_y':im.shape[0], 'bf_tif_slices':im.shape[2],
             'staining_x':ds.data.shape[2], 'staining_y':ds.data.shape[1], 'staining_slices':ds.data.shape[0],             
         'n_slices_txt_file':mri_slices.shape[0]}
    else:
        
    
        mri = nb.load('/home/public/HISTO_TO_MRI_1/MRI_DATA/MRI/PD/FLASH/{subject_id}/{subject_id}_pd_flash_006_ts.nii'.format(**locals())).get_data()

        r = {'subject_id':subject_id, 'bf_tif_x':im.shape[2], 'bf_tif_y':im.shape[1], 'bf_tif_slices':im.shape[0],
             'mri_x':mri.shape[0], 'mri_y':mri.shape[1], 'mri_n_slices':mri.shape[2],
             'staining_x':ds.data.shape[2], 'staining_y':ds.data.shape[1], 'staining_slices':ds.data.shape[0],
             'n_slices_txt_file':mri_slices.shape[0]}
    
    results.append(r)

13044
/home/mkeuken1/data/post_mortem/new_data_format/13044/images.hdf5
13046
/home/mkeuken1/data/post_mortem/new_data_format/13046/images.hdf5
13054
/home/mkeuken1/data/post_mortem/new_data_format/13054/images.hdf5
13058
/home/mkeuken1/data/post_mortem/new_data_format/13058/images.hdf5
13060
/home/mkeuken1/data/post_mortem/new_data_format/13060/images.hdf5
13061
/home/mkeuken1/data/post_mortem/new_data_format/13061/images.hdf5
13072
/home/mkeuken1/data/post_mortem/new_data_format/13072/images.hdf5
13074
/home/mkeuken1/data/post_mortem/new_data_format/13074/images.hdf5


In [233]:
from scipy import ndimage
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

In [ ]:

for modality in ['T1', 'T2star', 'QSM'][0:]:
    for subject_id in subject_ids:

        print modality, subject_id

        pdf = PdfPages('/home/mkeuken1/data/post_mortem/visualize_stains_v1/{modality}_stain_reg_{subject_id}_v2.pdf'.format(**locals()))

        year = '20' + str(subject_id)[:2]
        n = str(subject_id)[2:]
        
        mri_slices = pandas.read_csv('/home/public/HISTO_TO_MRI_1/Blockface_stacks/True_slice_distance/{n}_{year}_3removed.txt'.format(**locals()), header=None, names=['slice'])
        mri_slices['slice'] = mri_slices.slice.apply(lambda e: e / 50 * 50)
        mri_slices = mri_slices.set_index('slice')
       
        mri_slices['n'] = np.arange(mri_slices.shape[0])
        
        ds = pystain.StainDataset(subject_id)
      
        if modality == 'T1':
            mri = nb.load('/home/public/HISTO_TO_MRI_1/MRI_DATA/MRI/PD/T1_map/{subject_id}/{subject_id}_T1_2_flash_006_ts.nii.gz'.format(**locals())).get_data()
            
        elif modality == 'T2star':
            mri = nb.load('/home/public/HISTO_TO_MRI_1/MRI_DATA/MRI/PD/T2s_map/{subject_id}/{subject_id}_T2map_2_flash_006_ts.nii.gz'.format(**locals())).get_data()

        elif modality == 'QSM':
            mri = nb.load('/home/public/HISTO_TO_MRI_1/MRI_DATA/MRI/PD/QSM/{subject_id}/{subject_id}_qsm_2_flash_ts.nii.gz'.format(**locals())).get_data()            

        mri = np.swapaxes(mri, 0, 1)
        im = skimage.io.imread('/home/public/HISTO_TO_MRI_1/Blockface_stacks/{subject_id}/{subject_id}_rt.tif'.format(**locals()))

        if mri.shape[2] != im.shape[0]:
            im = im[3:, ...]

        assert(mri.shape[2] == im.shape[0])

        if subject_id in [13044]:
            mri = np.rot90(mri, 3)

        if subject_id in [13046]:
            mri = np.rot90(mri, 1)
        if subject_id in [13054]:
            mri = np.rot90(mri, 3)
        if subject_id in [13058]:
            mri = np.rot90(mri, 1)
        if subject_id in [13060]:
            mri = np.rot90(mri, 1)
        if subject_id in [13061]:
            mri = np.rot90(mri, 3)
        if subject_id in [13072]:
            mri = np.rot90(mri, 1)
        if subject_id in [13074]:
            mri = np.rot90(mri, 3)
      
        try:
            assert(mri.shape[-1] == mri_slices.shape[0])

            print 'MRI Shape: %s' % list(mri.shape)
            print 'Stain shape: %s:' %  list(ds.data.shape)
            print 'Blockface shape: %s:' % list(im.shape)

            mri_in_stain_space = np.zeros((len(ds.slices), ds.data.shape[1], ds.data.shape[2]))
            blockface_image = np.zeros((len(ds.slices), ds.data.shape[1], ds.data.shape[2], 3))

            for slice in ds.slices[:]:
                print slice

                if slice in mri_slices.index:

                    mri_slice_idx = mri_slices.ix[slice].n
                    stain_slice_idx = ds._get_index_slice(slice)

                    plt.subplot(141)
                    plt.imshow(sp.ndimage.gaussian_filter(ds.data[stain_slice_idx, ..., ds._get_index_stain('SMI32')], 7.5), cmap=plt.cm.inferno)
                    plt.contour(ds.thresholded_mask[stain_slice_idx, ...] == False, levels=[.5], colors=['white'])

                    plt.xticks(np.arange(0, plt.xticks()[0][-1], 250))
                    plt.yticks(np.arange(0, plt.yticks()[0][-1], 250))
                    plt.title('SMI32')

                    plt.subplot(142)
                    plt.imshow(sp.ndimage.gaussian_filter(ds.data[stain_slice_idx, ..., ds._get_index_stain('PARV')], 7.5), cmap=plt.cm.inferno)
                    plt.contour(ds.thresholded_mask[stain_slice_idx, ...] == False, levels=[.5], colors=['white'])

                    plt.xticks(np.arange(0, plt.xticks()[0][-1], 250))
                    plt.yticks(np.arange(0, plt.yticks()[0][-1], 250))    
                    plt.title('PARV')    

                    plt.subplot(143)

                    if modality == 'T2star':
                        plt.imshow(mri[:, :, mri_slice_idx], cmap=plt.cm.inferno, vmin=0, vmax=65)
                    else:
                        plt.imshow(mri[:, :, mri_slice_idx], cmap=plt.cm.inferno)
                    # plt.axis('off')
                    plt.contour(ds.thresholded_mask[stain_slice_idx, ...] == False, levels=[.5], colors=['white'])
                    plt.xticks(np.arange(0, plt.xticks()[0][-1], 250))
                    plt.yticks(np.arange(0, plt.yticks()[0][-1], 250))
                    plt.title(modality)    


                    plt.subplot(144)
                    plt.imshow(im[mri_slice_idx, ...])
                    # plt.axis('off')
                    plt.contour(ds.thresholded_mask[stain_slice_idx, ...] == False, levels=[.5], colors=['white'])
                    plt.xticks(np.arange(0, plt.xticks()[0][-1], 250))
                    plt.yticks(np.arange(0, plt.yticks()[0][-1], 250))
                    plt.title('Blockface image')                

                    plt.gcf().set_size_inches(40, 20)
                    plt.suptitle('Slice %d' % slice)
                    plt.savefig(pdf, format='pdf')
                    plt.close(plt.gcf())

                    mri_in_stain_space[stain_slice_idx, ...] = mri[:, :, mri_slice_idx]
                    blockface_image[stain_slice_idx, ...] = im[mri_slice_idx, ...]

            if '{modality}_in_stain_space'.format(**locals()) in ds.h5file.keys():
                del ds.h5file['{modality}_in_stain_space'.format(**locals())]

            ds.h5file['{modality}_in_stain_space'.format(**locals())] = mri_in_stain_space
            ds.h5file['blockface_image'] = blockface_image
            ds.h5file.flush()


        except Exception as e:
            print e

        pdf.close()

T1 13044
/home/mkeuken1/data/post_mortem/new_data_format/13044/images.hdf5
MRI Shape: [2240, 1768, 49]
Stain shape: [29, 2240, 1768, 12]:
Blockface shape: [49, 2240, 1768, 3]:
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
T1 13046
/home/mkeuken1/data/post_mortem/new_data_format/13046/images.hdf5
MRI Shape: [2072, 1488, 48]
Stain shape: [26, 2072, 1488, 12]:
Blockface shape: [48, 2072, 1488, 3]:
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
T1 13054
/home/mkeuken1/data/post_mortem/new_data_format/13054/images.hdf5
MRI Shape: [1944, 1888, 50]
Stain shape: [23, 1944, 1888, 12]:
Blockface shape: [50, 1944, 1888, 3]:
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
T1 13058
/home/mkeuken1/data/post_mortem/new_data_format/13058/images.hdf5
